British Airways is a flag carrier airline,it is the primary airline of the United Kingdom and internationally represents it.

## Web Scraping

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways/page/{}/?sortby=post_date%3ADesc&pagesize=100"


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL without page number
base_url = "https://www.airlinequality.com/airline-reviews/british-airways/page/{}/?sortby=post_date%3ADesc&pagesize=100"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Prepare a list to store reviews
data = []

# Loop through pages (adjust the range as needed to cover all pages)
for page in range(1, 11):  # Assuming there are 10 pages; adjust accordingly
    print(f"Scraping Page: {page}")
    url = base_url.format(page)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all review articles
    reviews = soup.find_all('article', itemprop='review')

    if not reviews:
        print(f"No reviews found on page {page}. Ending scrape.")
        break

    for review in reviews:
        # Extract title and remove quotes
        title = review.find('h2', class_='text_header')
        title = title.get_text(strip=True).replace('"', '').replace("'", '') if title else 'No Title'

        # Extract content, remove the green tick and 'Trip Verified|'
        content = review.find('div', class_='text_content')
        content = content.get_text(strip=True).replace('✅', '').replace('Trip Verified|', '').strip() if content else 'No Content'

        # Create 'Trip Verified' feature
        trip_verified = 'yes' if 'Trip Verified' in content else 'no'

        # Extract overall rating
        rating_div = review.find('div', itemprop='reviewRating')
        rating = rating_div.get('class', [''])[0].replace('rating-', '') if rating_div else 'No Rating'

        # Extract detailed stats
        stats_table = review.find('table', class_='review-ratings')
        stats = {
            "Aircraft": "",
            "Type Of Traveller": "",
            "Seat Type": "",
            "Route": "",
            "Date Flown": "",
            "Seat Comfort": "",
            "Cabin Staff Service": "",
            "Food & Beverages": "",
            "Inflight Entertainment": "",
            "Ground Service": "",
            "Wifi & Connectivity": "",
            "Value For Money": "",
            "Recommended": ""
        }

        if stats_table:
            for row in stats_table.find_all('tr'):
                columns = row.find_all('td')
                if len(columns) == 2:
                    key = columns[0].get_text(strip=True)

                    # Handling star ratings
                    stars = columns[1].find_all('span', class_='star fill')
                    value = str(len(stars)) if stars else columns[1].get_text(strip=True)

                    if key in stats:
                        stats[key] = value

        # Split route into start and end route
        start_route, end_route = (stats["Route"].split(" to ") + [""])[:2]
        stats["Start Route"] = start_route
        stats["End Route"] = end_route

        # Remove the original 'Route' key
        stats.pop("Route", None)

        # Append data
        data.append({
            "Title": title,
            "Content": content,
            "Trip Verified": trip_verified,
            "Overall Rating": rating,
            **stats
        })

# Convert to DataFrame
df = pd.DataFrame(data)




Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10


In [6]:
df.head()

,Title,Content,Trip Verified,Overall Rating,Aircraft,Type Of Traveller,Seat Type,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Start Route,End Route
0,managed to get everything right,This time British Airways managed to get every...,no,10,Boeing 777,Business,Premium Economy,March 2025,5,5,4,4,5,4,5,yes,Gatwick,Antigua
1,The seats were excellent,"The seats were excellent, with a feel of much ...",no,10,,Couple Leisure,Premium Economy,February 2025,5,4,3,5,4,,5,yes,Mumbai,London
2,the crew were awful,After the nightmare of getting through triple ...,no,10,A350,Couple Leisure,Business Class,February 2025,4,1,2,3,4,,3,no,Nairobi,London
3,not use British Airways on this route,Prior to boarding a gate agent seemed to pick ...,no,10,A320,Solo Leisure,Economy Class,February 2025,4,1,,,2,,2,no,London Gatwick,Marrakech
4,they still havent replied,I flew from Amsterdam to Las Vegas with a layo...,no,10,,Business,Premium Economy,November 2024,3,3,3,3,1,,1,no,Amsterdam,Las Vegas via London


In [7]:
duplicate_count = df.duplicated(subset='Content').sum()

print("Number of duplicate reviews:", duplicate_count)

Number of duplicate reviews: 0


## Cleaning Data

In [9]:
df.to_csv('C:/Users/vadre/OneDrive/Desktop/britishairwaysdata.csv', index=False)



In [10]:
# Create 'Trip Verified' feature with 'yes' or 'no'
df['Trip Verified'] = df['Content'].apply(lambda x: 'yes' if 'Trip Verified' in x else 'no')

# Remove 'Trip Verified|' from 'Content' and strip spaces
df['Content'] = df['Content'].str.replace('Trip Verified|', '', regex=False).str.strip()


In [11]:
df.head()

,Title,Content,Trip Verified,Overall Rating,Aircraft,Type Of Traveller,Seat Type,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Start Route,End Route
0,managed to get everything right,This time British Airways managed to get every...,no,10,Boeing 777,Business,Premium Economy,March 2025,5,5,4,4,5,4,5,yes,Gatwick,Antigua
1,The seats were excellent,"The seats were excellent, with a feel of much ...",no,10,,Couple Leisure,Premium Economy,February 2025,5,4,3,5,4,,5,yes,Mumbai,London
2,the crew were awful,After the nightmare of getting through triple ...,no,10,A350,Couple Leisure,Business Class,February 2025,4,1,2,3,4,,3,no,Nairobi,London
3,not use British Airways on this route,Prior to boarding a gate agent seemed to pick ...,no,10,A320,Solo Leisure,Economy Class,February 2025,4,1,,,2,,2,no,London Gatwick,Marrakech
4,they still havent replied,I flew from Amsterdam to Las Vegas with a layo...,no,10,,Business,Premium Economy,November 2024,3,3,3,3,1,,1,no,Amsterdam,Las Vegas via London


In [12]:
# Remove quotes from 'Title'
df['Title'] = df['Title'].str.replace('"', '').str.replace("'", '').str.strip()


In [13]:
df.head()

,Title,Content,Trip Verified,Overall Rating,Aircraft,Type Of Traveller,Seat Type,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Start Route,End Route
0,managed to get everything right,This time British Airways managed to get every...,no,10,Boeing 777,Business,Premium Economy,March 2025,5,5,4,4,5,4,5,yes,Gatwick,Antigua
1,The seats were excellent,"The seats were excellent, with a feel of much ...",no,10,,Couple Leisure,Premium Economy,February 2025,5,4,3,5,4,,5,yes,Mumbai,London
2,the crew were awful,After the nightmare of getting through triple ...,no,10,A350,Couple Leisure,Business Class,February 2025,4,1,2,3,4,,3,no,Nairobi,London
3,not use British Airways on this route,Prior to boarding a gate agent seemed to pick ...,no,10,A320,Solo Leisure,Economy Class,February 2025,4,1,,,2,,2,no,London Gatwick,Marrakech
4,they still havent replied,I flew from Amsterdam to Las Vegas with a layo...,no,10,,Business,Premium Economy,November 2024,3,3,3,3,1,,1,no,Amsterdam,Las Vegas via London


In [14]:
df.to_csv('C:/Users/vadre/OneDrive/Desktop/data.csv', index=False)
